<a href="https://colab.research.google.com/github/nhyunjeong/bird-classification-/blob/main/%5BObject_Detection%5D_birds_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EB%B6%84%EB%A5%98_(yolov4)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 드라이브 GPU 버전 확인하기 
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-12ce6e50-bf9d-b6cd-1e68-340dd12dcef4)


사전 확인사항 
1. 라벨링한 이미지 .zip 파일 업로드하기 (코랩에)
2.  구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



3.yolo_custom_model_Training3 폴더 생성하기 (mkdir) 

- yolo_custom_model_Training3 폴더 안에 라벨링한 이미지 .zip 위치 이동하기 (드라이브에서 직접 넣는게 편함) 

In [ ]:
%cd /content/drive/MyDrive
!mkdir yolo_custom_model_Training3

/content/drive/MyDrive
mkdir: cannot create directory ‘yolo_custom_model_Training3’: File exists


4. 만든 폴더 위치로 이동하기
- 가져다 놓은 zip 파일 있는지 ls 로 확인하기 

In [ ]:
#'/content/drive/MyDrive/yolo_custom_model_Training3' 밑에 어떤 것들이 있는지 확인하기(처음엘 라벨링된 zip 파일만 존재 )
!ls '/content/drive/MyDrive/yolo_custom_model_Training3'


all2_birds.zip	yolov4.conv.137


5. /content/drive/MyDrive/yolo_custom_model_Training3' 로 이동하여 custom_data 라는 폴더 생성

In [ ]:
%cd '/content/drive/MyDrive/yolo_custom_model_Training3' 
!mkdir custom_data 

/content/drive/MyDrive/yolo_custom_model_Training3


6. 라벨링된zip파일의 압축을 custom_data 폴더 밑에 압축해제하기

In [ ]:
!unzip '/content/drive/MyDrive/yolo_custom_model_Training3/all2_birds.zip' -d '/content/drive/MyDrive/yolo_custom_model_Training3/custom_data'

Archive:  /content/drive/MyDrive/yolo_custom_model_Training3/all2_birds.zip
  inflating: /content/drive/MyDrive/yolo_custom_model_Training3/custom_data/1.jpg  
  inflating: /content/drive/MyDrive/yolo_custom_model_Training3/custom_data/1.txt  
  inflating: /content/drive/MyDrive/yolo_custom_model_Training3/custom_data/10.jpg  
  inflating: /content/drive/MyDrive/yolo_custom_model_Training3/custom_data/10.txt  
  inflating: /content/drive/MyDrive/yolo_custom_model_Training3/custom_data/100.jpg  
  inflating: /content/drive/MyDrive/yolo_custom_model_Training3/custom_data/100.txt  
  inflating: /content/drive/MyDrive/yolo_custom_model_Training3/custom_data/101.jpg  
  inflating: /content/drive/MyDrive/yolo_custom_model_Training3/custom_data/101.txt  
  inflating: /content/drive/MyDrive/yolo_custom_model_Training3/custom_data/102.jpg  
  inflating: /content/drive/MyDrive/yolo_custom_model_Training3/custom_data/102.txt  
  inflating: /content/drive/MyDrive/yolo_custom_model_Training3/custom

7. 현재 디렉토리 확인하기 

/content/drive/MyDrive/yolo_custom_model_Training3 여기여야함 

In [ ]:
%pwd

'/content/drive/MyDrive/yolo_custom_model_Training3'

8. 다크넷 신경망 다운 받기 

- object detection 을 전문으로 하는 다크넷 (신경망) 다운받기 
- got 허브 주소 입력 -> 다운받을 경로 입력 
- 나중에 폴더 확인해보면 여러가지 폴더가 많이 생긴것 확인 가능 

In [ ]:
# download dataset in current directory(above path)
!git clone 'https://github.com/AlexeyAB/darknet.git' '/content/drive/MyDrive/yolo_custom_model_Training3/darknet'

Cloning into '/content/drive/MyDrive/yolo_custom_model_Training3/darknet'...
remote: Enumerating objects: 15460, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 15460 (delta 6), reused 25 (delta 3), pack-reused 15424
Receiving objects: 100% (15460/15460), 14.09 MiB | 8.11 MiB/s, done.
Resolving deltas: 100% (10372/10372), done.
Checking out files: 100% (2053/2053), done.


9. 현재 디렉토리에서 다운 받은 다크넷 폴더로 이동하기 

In [ ]:
# move current directory to darknet
%cd /content/drive/MyDrive/yolo_custom_model_Training3/darknet

/content/drive/MyDrive/yolo_custom_model_Training3/darknet


10. makefile 내용 수정하기 

- darknet 파일에 들어가보면 makefile 이라는 폴더가 있는데 그 내용을 수정해 주어야합니다.
- 클릭하여 들어가보면 GPU = 0 , CUDNN = 0 등 설정이 되어있는데 0 이라는 의미는 사용하지 않겠다는 의미입니다.
- 이것을 사용하겠다는의미인 1로 변경해주어야합니다. 
- 리눅스 vi 편집기 명령어로 변경합니다.  

In [ ]:
# change setting values in 'Makefile' file 
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile # opencv 사용
!sed -i 's/GPU=0/GPU=1/' Makefile       # gpu 사용 

# cuda ? GPU 를 현재 피씨에서 사용할 수 있도록하는 펌웨어(하드웨어를 작동시키기 위한 소프트웨어)
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile   # cuda 를 사용가능하게하는것 
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile # cuda 를 사용가능하게 하는것 

!sed -i 's/LIBSO=0/LIBSO=1/' Makefile   # Library 관련한 파라미터 

11. 수정된 make 파일을 compile (실행) 환경설정을 하기 
- 변경된 내용을 반영하는것 


In [ ]:
# Compile model
"""  take care do not disconnect : file directory may be interupted 
if your network down during compile, I recommend delete darknet folder and restart number 4(get AlexeyAB/darknet)"""

!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old ==

12 darknet 적용 하기 

- 상위디렉토리로 이동하여 다운받은 다크넷 신경망 setting 하기 

- 성공적으로 수행 되었을때 나타나는 메세지

/content/drive/My Drive/yolo_custom_model_Training3

usage: darknet/darknet <function>  

In [ ]:
%cd ..
!darknet/darknet

/content/drive/MyDrive/yolo_custom_model_Training3
usage: darknet/darknet <function>


13. 사물검출에 필요한 코드 다운받기 (github) 주소 

- 오브젝터 디텍션에 필요한 파이썬 코드 4개이다. 

In [ ]:
!git clone 'https://github.com/jakkcoder/training_yolo_custom_object_detection_files' '/content/drive/MyDrive/yolo_custom_model_Training3/training_yolo_custom_object_detection_files-main'

Cloning into '/content/drive/MyDrive/yolo_custom_model_Training3/training_yolo_custom_object_detection_files-main'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


14. 코드를 다운 받은 경로로 이동하기 

In [ ]:
%cd /content/drive/MyDrive/yolo_custom_model_Training3/training_yolo_custom_object_detection_files-main

/content/drive/MyDrive/yolo_custom_model_Training3/training_yolo_custom_object_detection_files-main


15. ls 수행하여 4개의 파일이 존재하는지 확인하기 

  1. creating-files-data-and-name.py
  2. creating-train-and-test-txt-files.py
  3. 'Custom Object detection live video.ipynb'
  4. Rename_files.ipynb

  - 역할 : 훈련시키기 전에 파일들의 이름을 바꾸고 변경하는 코드들 
  

In [ ]:
# check out current dir files
!ls

 creating-files-data-and-name.py
 creating-train-and-test-txt-files.py
'Custom Object detection live video.ipynb'
 Rename_files.ipynb


16. 이미지와 라벨링된 텍스트가 있는 폴더(현재 custom_data)  로 위의 4개 파일중 .py file 2개를 복사합니다. 

- 훈련 데이터와 테스트 데이터파일의 목록(리스트)을 만드는 파일들임.

In [ ]:
# copy creating-train-and-test-txt-files.py & creating-files-data-and-name.py
"""creating-train-and-test-txt-files.py >> create 'train.txt' & 'test.txt' files
   creating-files-data-and-name.py >> create label 'labelled_data.data' file
   <if you excute both .py files, you get mentioned files upper lines 2,3>"""

!cp creating-train-and-test-txt-files.py /content/drive/MyDrive/yolo_custom_model_Training3/custom_data
!cp creating-files-data-and-name.py /content/drive/MyDrive/yolo_custom_model_Training3/custom_data

17. 이미지와 라벨링 된 텍스트가 있는 폴더로 이동하기 
- Parakeet_imgdata (폴더) 

In [ ]:
%cd /content/drive/MyDrive/yolo_custom_model_Training3/custom_data

/content/drive/MyDrive/yolo_custom_model_Training3/custom_data


18. 2개의 파이썬 코드의 내용 수정하기 

- 참고로 폴더명 잘 확인하기 

In [ ]:
# change paths in both .py files
# 리눅스 명령어로 creating-train-and-test-txt-files.py 의 내용중 39 번째 라인에있는 디렉토리를 @다음인 custom_data 로 변경
!sed -i '39 s@/home/my_name/Downloads/video-to-annotate@custom_data@' creating-train-and-test-txt-files.py
# jpeg 를 현재파일형식인 jpg로 바꿈
!sed -i '74 s@jpeg@jpg@' creating-train-and-test-txt-files.py
# home/my_name/Downloads/video-to-annotate 을 custom_data 로 변경 
!sed -i '36 s@/home/my_name/Downloads/video-to-annotate@custom_data@' creating-files-data-and-name.py

19. 상위 디렉토리로 이동하기 

In [ ]:
%cd ..

/content/drive/MyDrive/yolo_custom_model_Training3


20.  creating-train-and-test-txt-files.py 를 수행하여   
 훈련 데이터들(train.txt)목록과 테스트 데이터(test.txt)들의 목록을 생성합니다. 
 - 즉. 훈련과 테스트를 나누는 것임. (본인이 스스로 랜덤으로 만들어 냄) 

In [ ]:
# excute .py file >> 'train.txt', 'test.txt'   
!python custom_data/creating-train-and-test-txt-files.py

21. 정답데이터 (사랑앵무 ) 를 생성하기 
- 훈련데이터 목록, 테스트데이터 목록, class 이름이 어디있는지 목록정보를 만들어 주는 py 파일을 실행한다. 

- 실행이 완료 된 뒤 custom_data 파일로 들어가 보면 laballed_data.data 라는 파일이 생성된것을 확인 할 수 있다. 

클릭해서 들어가보면 

classes = 2   
train = custom_data/train.txt   
valid = custom_data/test.txt   
names = custom_data/classes.names   
backup = backup   

의 내용들이 존재

In [ ]:
# excute .py file >> 'labelled_data.data'
!python custom_data/creating-files-data-and-name.py

In [ ]:
%pwd

'/content/drive/MyDrive/yolo_custom_model_Training3'

**yolov4.conv.137 다운받기(google검색) github** 

- yolo의 convolution 층 
- /content/drive/MyDrive/yolo_custom_model_Training3 위치에다가 파일을 가져다 둡니다. 

22. 현재 디렉토리에 학습 시킨 후의 가중치를 저장할 **custom_weight** 폴더 생성하기 

In [ ]:
# create directory 'custom_weight'
!mkdir custom_weight

23. 현재 디렉토리 확인하기 

- /content/drive/My Drive/yolo_custom_model_Training3  <-- 여기면 맞습니다.

In [ ]:
%pwd

'/content/drive/MyDrive/yolo_custom_model_Training3'

***
24. 현재 디렉토리의 목록 확인하기 

-rw-------  1 root root    298344 Jul 14 05:44 bird_train_200.zip   
drwx------  2 root root      4096 Jul 14 06:36 custom_data/   
drwx------  2 root root      4096 Jul 14 06:48 **custom_weight/**   
drwx------ 16 root root      4096 Jul 14 06:20 darknet/   
drwx------  3 root root      4096 Jul 14 06:21    training_yolo_custom_object_detection_files-main/   
-rw-------  1 root root 170038676 May  3 03:03 **yolov4.conv.137**

- yolov4.conv.137 을 방금 만든 coustom_weight 파일에 이동하기 

In [ ]:
%ls -l

total 170656
-rw-------  1 root root   4696189 Jul 19 07:37 all2_birds.zip
drwx------  2 root root      4096 Jul 19 07:43 custom_data/
drwx------  2 root root      4096 Jul 19 07:43 custom_weight/
drwx------ 16 root root      4096 Jul 19 07:41 darknet/
drwx------  3 root root      4096 Jul 19 07:42 training_yolo_custom_object_detection_files-main/
-rw-------  1 root root 170038676 May  3 03:03 yolov4.conv.137


25. 28. yolov4.conv.137 모델을 custom_weight 밑에 이동합니다. 

In [ ]:
# move 'yolov4.conv.137' file to 'custom_weight' dir
!mv /content/drive/MyDrive/yolo_custom_model_Training3/yolov4.conv.137 custom_weight/

27. /content/drive/MyDrive/yolo_custom_model_Training3/darknet/cfg 밑으로 이동합니다.

- 다크넷 폴더 밑 cfg 밑에 yolo 의 환경 구성 파일들이 버전별로 존재한다 -> 그쪽으로 리눅스 명령어로 복사하는것 (object detection 을 구성하기 위한 것들 ) 

- custom.cfg 가 생성된것을 볼 수 있다. 
- 나중에 이것을 다운받게 될 것 



In [ ]:
%cd /content/drive/MyDrive/yolo_custom_model_Training3/darknet/cfg

/content/drive/MyDrive/yolo_custom_model_Training3/darknet/cfg


28. yolov4.cfg를  yolov4_custom.cfg 로 복사합니다.

- 나중에 yolov4_custom.cfg 다운 받을것! 

In [ ]:
# copy yolov4.cfg file & rename & paste
!cp yolov4.cfg yolov4_custom.cfg

**이제부터는 yolov4_custom.cfg 내용을 클래스 갯수와 이런저런 변수사항 생각하면서 수정해야함 !!!!**


29. 학습중 메모리 초과로 학습이 중단되는것을 막기 위해 다음과 같이 배치 사이즈를 조정합니다. 

In [ ]:
# change values for training
!sed -i '2 s@batch=64@batch=8@' yolov4_custom.cfg # 64 로 돌리면 메모리 초과오류 날 수 있음 8로 변경 

!sed -i '7 s@width=608@width=416@' yolov4_custom.cfg # 이미지의 가로 세로 변경 
!sed -i '8 s@height=608@height=416@' yolov4_custom.cfg  

!sed -i '19 s@500500@12000@' yolov4_custom.cfg  #maxbatch = class*2000 (클래스갯수 * 2000 )
!sed -i '21 s@400000,450000@9600,10800@' yolov4_custom.cfg  # maxbatch*0.8, maxbatch*0.9

# 기존코드에서는 클래스 갯수 80개 변경할 클래수 갯수는 6개 
!sed -i '968 s@classes=80@classes=6@' yolov4_custom.cfg
!sed -i '1056 s@classes=80@classes=6@' yolov4_custom.cfg
!sed -i '1144 s@classes=80@classes=6@' yolov4_custom.cfg

# 합성곱시 쓰이는 filter 의 갯수 
!sed -i '961 s@filters=255@filters=33@' yolov4_custom.cfg  #filters=(4+1+classes)*3 
!sed -i '1049 s@filters=255@filters=33@' yolov4_custom.cfg
!sed -i '1137 s@filters=255@filters=33@' yolov4_custom.cfg

벡업 디렉토리를 생성하기 위해 디렉토리 이동하기 
30. 폴더 위치 이동하기 /content/drive/MyDrive/yolo_custom_model_Training3 로 이동합니다.


In [ ]:
%cd /content/drive/MyDrive/yolo_custom_model_Training3

/content/drive/MyDrive/yolo_custom_model_Training3


31. 학습될 가중치를 저장할 backup 폴더 생성

In [ ]:
!mkdir backup

32. /content/drive/MyDrive/yolo_custom_model_Training3 로 이동합니다.

In [ ]:
%cd /content/drive/MyDrive/yolo_custom_model_Training3

/content/drive/MyDrive/yolo_custom_model_Training3


33. 다크넷을 이용하여 학습 데이터 훈련 시키기 

학습시 필요한 파일은   
1. labelled_data.data  
2. yolov4_custom.cfg
3. yolov4.conv.137

In [ ]:
!darknet/darknet detector train custom_data/labelled_data.data darknet/cfg/yolov4_custom.cfg custom_weight/yolov4.conv.137 -dont_show

/bin/bash: darknet/darknet: Permission denied


**후에 반드시 로컬로 다운받아야 할 가중치와 cfg 파일 위치 !!!!**

1. yolov4_custom.cfg    
  - darknet/cfg/yolov4_custom.dfg
2. 가중치!! (backup 밑에 있습니다. )    
  - backup/yolov4_custom_last.weights 

